В начале привычные вещи, ничего необычного

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim

import pandas as pd
import numpy as np

import tqdm
import matplotlib.pyplot as plt

import re

In [4]:
# Загружаем файлы и готовим некоторые списки

df = pd.read_csv('гороскопы мейла с фичами.csv', sep=';') # в этом файле есть координаты, но только по 2 первых слова прогноза

signs = ['Овен',	'Телец',	'Близнецы',	'Рак',	'Лев',	'Дева',	'Весы',	'Скорпион',	'Стрелец',	'Козерог',	'Водолей',	'Рыбы']
planets = {'Sun': 10,  'Mercury': 199, 'Venus': 299, 'Mars': 499, 'Jupiter': 599, 'Saturn': 699, 'Uranus': 799, 'Neptune': 899, 'Pluto': 999}
features = []
for k, v in planets.items():
  features += [k+'_x', k+'_y', k+'_z', k+'_vx', k+'_vy', k+'_vz', k+'_l', k+'_ry', k+'_rr']

In [5]:
df_orig = pd.read_csv('df_three.csv', sep=';') # это оригинальный файл с соскрапленными гороскопами

def conv_date(d):
  d = d.split()
  year = d[-1]
  day = d[0]
  if d[1] == 'января':
    month = '01'
  elif d[1] == 'февраля':
    month = '02'
  elif d[1] == 'марта':
    month = '03'
  elif d[1] == 'апреля':
    month = '04'
  elif d[1] == 'мая':
    month = '05'
  elif d[1] == 'июня':
    month = '06'
  elif d[1] == 'июля':
    month = '07'
  elif d[1] == 'августа':
    month = '08'
  elif d[1] == 'сентября':
    month = '09'
  elif d[1] == 'октября':
    month = '10'
  elif d[1] == 'ноября':
    month = '11'
  elif d[1] == 'декабря':
    month = '12'

  return '-'.join([year, month, day])

df_orig = df_orig[['date'] + signs].copy()
df_orig.fillna('0')
df_orig['date'] =  df_orig['date'].apply(lambda x: conv_date(x))
df_orig['date_for_epoch'] =  df_orig['date'].apply(lambda x: Time(x).jd)

start_sign_cols = []
for sign in signs:
  start_sign_cols.append('start_' + sign)
  df_orig['start_' + sign] = df_orig[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()[0:5]).lower() if type(x) == str else x)
  df_orig[sign] = df_orig[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()).lower() if type(x) == str else x)

In [6]:
df[start_sign_cols] = df_orig[start_sign_cols]

In [7]:
df_orig.head(2)

,date,Овен,Телец,Близнецы,Рак,Лев,Дева,Весы,Скорпион,Стрелец,Козерог,Водолей,Рыбы,date_for_epoch,start_Овен,start_Телец,start_Близнецы,start_Рак,start_Лев,start_Дева,start_Весы,start_Скорпион,start_Стрелец,start_Козерог,start_Водолей,start_Рыбы
0,2020-10-18,насыщенный интересный день за любые дела вы бе...,вам может быть нелегко отстоять свои интересы ...,тот самый день когда ваши поступки вызывают вс...,для самостоятельной работы это не самый подход...,вы без труда завоевываете симпатии новых знако...,первая половина дня проходит спокойно можно де...,благоприятный день для общения удается произве...,не спешите это не тот день когда можно за счит...,напряженный день будьте осторожны и внимательн...,с эмоциональной точки зрения день будет доволь...,утром стоит быть осторожнее поскольку в это вр...,здравый смысл и практичность помогают избежать...,2459140.5,насыщенный интересный день за любые,вам может быть нелегко отстоять,тот самый день когда ваши,для самостоятельной работы это не,вы без труда завоевываете симпатии,первая половина дня проходит спокойно,благоприятный день для общения удается,не спешите это не тот,напряженный день будьте осторожны и,с эмоциональной точки зрения день,утром стоит быть осторожнее поскольку,здравый смысл и практичность помогают
1,2020-10-17,будьте практичны этот день может положить нача...,день складывается удачно хотя порой вы сами се...,благоприятный день для любых встреч и перегово...,день подходит для решения любых вопросов в том...,вы способны на многое и сами это понимаете но ...,утро потребует осторожности в делах и сдержанн...,удачный день открываются новые возможности и в...,утро проходит очень плодотворно в это время вы...,вы решите многие проблемы если расстанетесь с ...,день складывается напряженно но влияние благоп...,неплохой день подходящий для любого общения вы...,день хоть и не лишен сложных моментов но все ж...,2459139.5,будьте практичны этот день может,день складывается удачно хотя порой,благоприятный день для любых встреч,день подходит для решения любых,вы способны на многое и,утро потребует осторожности в делах,удачный день открываются новые возможности,утро проходит очень плодотворно в,вы решите многие проблемы если,день складывается напряженно но влияние,неплохой день подходящий для любого,день хоть и не лишен


In [8]:
df.head(2)

,Unnamed: 0,date,date_for_epoch,start_Овен,start_Телец,start_Близнецы,start_Рак,start_Лев,start_Дева,start_Весы,start_Скорпион,start_Стрелец,start_Козерог,start_Водолей,start_Рыбы,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,...,Jupiter_vz,Jupiter_l,Jupiter_ry,Jupiter_rr,Saturn_x,Saturn_y,Saturn_z,Saturn_vx,Saturn_vy,Saturn_vz,Saturn_l,Saturn_ry,Saturn_rr,Uranus_x,Uranus_y,Uranus_z,Uranus_vx,Uranus_vy,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr
0,0,2020-10-18,2459140.5,насыщенный интересный день за любые,вам может быть нелегко отстоять,тот самый день когда ваши,для самостоятельной работы это не,вы без труда завоевываете симпатии,первая половина дня проходит спокойно,благоприятный день для общения удается,не спешите это не тот,напряженный день будьте осторожны и,с эмоциональной точки зрения день,утром стоит быть осторожнее поскольку,здравый смысл и практичность помогают,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.54520,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,...,-0.000161,0.029598,5.124750,0.015327,4.254547,-8.982140,-0.056410,0.011974,-0.012692,-0.000228,0.057403,9.938973,0.016597,14.631491,11.826922,-0.155757,0.005041,-0.012651,0.000043,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819
1,1,2020-10-17,2459139.5,будьте практичны этот день может,день складывается удачно хотя порой,благоприятный день для любых встреч,день подходит для решения любых,вы способны на многое и,утро потребует осторожности в делах,удачный день открываются новые возможности,утро проходит очень плодотворно в,вы решите многие проблемы если,день складывается напряженно но влияние,неплохой день подходящий для любого,день хоть и не лишен,-0.911909,-0.402126,0.000024,0.007225,-0.015684,3.191686e-07,0.005756,0.996636,-0.000283,-0.55172,-0.476378,-0.039085,0.007454,0.013131,0.002334,0.004216,0.729972,-0.014328,-1.145999,0.277093,0.022853,-0.011965,-0.022388,0.001016,0.006811,...,-0.000161,0.029509,5.109411,0.015350,4.242709,-8.969385,-0.056182,0.011702,-0.012818,-0.000228,0.057307,9.922379,0.016592,14.626586,11.839634,-0.155800,0.004768,-0.012774,0.000043,0.108687,18.818561,-0.004331,28.507044,-5.870516,-0.565436,0.007785,-0.012574,-0.000076,0.168130,29.110723,0.010160,12.924825,-31.607126,-0.662256,0.010184,-0.015076,-0.000926,0.197257,34.154064,0.017824


In [9]:
df = df.drop(['date',	'date_for_epoch'], axis=1)

In [10]:
df_orig = df_orig.dropna()

Тут все что касается эмбедингов.


In [11]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [12]:
data_part = [d.split() for d in list(df_orig[signs].values.reshape(3708))]
model = Word2Vec(data_part, min_count=1, size=50, workers=3, window =3, sg = 1)

# Тут как обычно модель готовится


In [13]:
# model['день'], model.most_similar('день'), model.similar_by_vector(model['день'])

In [ ]:
# model.save("word2vec_size_300_all_datasets.model")
# model = Word2Vec.load("word2vec.model")

In [34]:
# Создаем пары списков, в одном будут пятерки первых слов, во втором сумма пятерок векторов

data_part_5_vec = [np.array(model[el[0:5]]).sum(axis=0) for el in data_part]
data_part_5_words = [' '.join(el[0:5]) for el in data_part]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [36]:
# Создаем вторую модель на основе полученных списков суммированных векторов и пятерок слов
# Теперь по этой модели можно получать фразы, только в выходе гана тоже надо добавить суммирование векторов

from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
vector_length = 50
kv = WordEmbeddingsKeyedVectors(vector_length)
kv.add(data_part_5_words, data_part_5_vec)

In [29]:
kv.similar_by_vector(data_part_5_vec[0])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('насыщенный интересный день за любые', 1.0000001192092896),
 ('спокойный благоприятный день привычные дела', 0.9527862071990967),
 ('интересный день сидеть сложа руки', 0.9440630674362183),
 ('легкий удачный день приносящий блестящие', 0.9393801689147949),
 ('отличный плодотворный день даже какието', 0.9388961791992188),
 ('беспокойный насыщенный день заранее настройтесь', 0.9358505606651306),
 ('плодотворный день нет такого дела', 0.9339351058006287),
 ('удачный гармоничный день обещающий новые', 0.9335250854492188),
 ('удачный плодотворный день обещающий большие', 0.9326398968696594),
 ('насыщенный и плодотворный день подходящий', 0.9326186776161194)]

In [31]:
kv.similar_by_vector(np.random.uniform(size=50))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('за что бы вы ни', 0.2940722107887268),
 ('прежде чем браться за чтото', 0.2681558430194855),
 ('за какие бы дела вы', 0.2677598297595978),
 ('прежде чем браться за новые', 0.26385533809661865),
 ('вы понимаете что дела требуют', 0.25306960940361023),
 ('не спешите прежде чем браться', 0.2526654005050659),
 ('не бойтесь браться за решение', 0.2470303624868393),
 ('какие бы цели вы ни', 0.24271449446678162),
 ('какие бы цели вы ни', 0.24271449446678162),
 ('что бы ни происходило вы', 0.2367827296257019)]

In [32]:
kv.similar_by_vector(np.random.uniform(size=50))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('вы с удовольствием беретесь за', 0.256215900182724),
 ('прежде чем браться за новые', 0.252946674823761),
 ('вы понимаете что дела требуют', 0.2518504858016968),
 ('за что бы вы ни', 0.23862135410308838),
 ('вы ставите перед собой очень', 0.23661071062088013),
 ('за какие бы дела вы', 0.22619186341762543),
 ('вы прекрасно ладите с людьми', 0.2229272425174713),
 ('прежде чем браться за чтото', 0.22242560982704163),
 ('вы делаете именно то что', 0.22153116762638092),
 ('вы отлично справляетесь с собственными', 0.22126413881778717)]

In [33]:
kv.similar_by_vector(np.random.uniform(size=50))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('не спешите браться за новые', 0.3398624658584595),
 ('не спешите браться за новые', 0.3398624658584595),
 ('утром не стоит браться за', 0.32027190923690796),
 ('не бойтесь браться за решение', 0.3170746862888336),
 ('за какие бы дела вы', 0.3107120394706726),
 ('не стоит браться за важные', 0.3092949390411377),
 ('вы готовы взяться за сложные', 0.30812788009643555),
 ('за что бы вы ни', 0.307669460773468),
 ('вы с удовольствием беретесь за', 0.2963026463985443),
 ('можно браться за сложные масштабные', 0.2946898341178894)]

In [41]:
test_phrase = 'непростой день будет простым и'
test_vec = np.array([model[word] for word in test_phrase.split(' ')]).sum(axis=0)
kv.similar_by_vector(test_vec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('день будет ярким запоминающимся и', 0.986470103263855),
 ('день будет полным переживаний и', 0.9838017821311951),
 ('день будет беспокойным и полным', 0.9836120009422302),
 ('день будет полным событий и', 0.983271598815918),
 ('день будет противоречивым беспокойным и', 0.9785866141319275),
 ('день будет непростым и довольно', 0.9777127504348755),
 ('день будет беспокойным и насыщенным', 0.9770214557647705),
 ('день будет насыщенным и довольно', 0.9752289652824402),
 ('день будет насыщенным и довольно', 0.9752289652824402),
 ('день будет насыщенным и довольно', 0.9752289652824402)]